In [1]:
##### Imports #####
import pandas as pd
from scripts import functions

2023-05-31 10:09:31.938506: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
##### Constants #####
HUMAN_CHROMOSOMES = ('chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX', 'chrY', 'chrM')
GENCODE_v43_ANNOTATIONS_FILE = '/datasets2/genomes/annotations/hg38.gencode.comprehensive.v43.annotation.gtf'

In [4]:
##### Settings #####
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows', 200)

In [5]:
##### Loading Data #####
repeat_regions = pd.read_csv('data/UCSC_hg38_simple_repeats_start=-1.tsv', sep='\t')
introns = pd.read_csv('data/gencode.v43.intron_coords.tsv', sep='\t')
print(repeat_regions.info())
print(repeat_regions.iloc[:3])
print(introns.info())
print(introns.iloc[:3])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1049715 entries, 0 to 1049714
Data columns (total 17 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   #bin           1049715 non-null  int64  
 1   chrom          1049715 non-null  object 
 2   chromStart     1049715 non-null  int64  
 3   chromEnd       1049715 non-null  int64  
 4   name           1049715 non-null  object 
 5   period         1049715 non-null  int64  
 6   copyNum        1049715 non-null  float64
 7   consensusSize  1049715 non-null  int64  
 8   perMatch       1049715 non-null  int64  
 9   perIndel       1049715 non-null  int64  
 10  score          1049715 non-null  int64  
 11  A              1049715 non-null  int64  
 12  C              1049715 non-null  int64  
 13  G              1049715 non-null  int64  
 14  T              1049715 non-null  int64  
 15  entropy        1049715 non-null  float64
 16  sequence       1049715 non-null  object 
dtypes: float

In [6]:
# Consensus size = Sequence length
repeat_regions = (repeat_regions.set_axis(['', 'Chrom', 'Start', 'End', '', 'Period', 'Copy_num', 'Consensus_size', '', '', '', '', '', '', '', '', 'Sequence'], axis=1)
								.drop([''], axis=1)
)
repeat_regions = repeat_regions.assign(Start = repeat_regions.Start + 1, 
                                    # ID = repeat_regions.Chrom + '_' + repeat_regions.Start.astype(str) + '_' + repeat_regions.End.astype(str) + '_' + repeat_regions.Consensus_size.astype(str) + '_' + repeat_regions.index.astype(str))
                                    ID = repeat_regions.Chrom + '_' + repeat_regions.Start.astype(str) + '_' + repeat_regions.End.astype(str) + '_' + repeat_regions.index.astype(str))
# repeat_regions.loc[repeat_regions.ID=='chr7_62175782_62178458_342', 'ID'] += '_' + repeat_regions.loc[repeat_regions.ID=='chr7_62175782_62178458_342'].index.astype(str)	#Problem child
repeat_regions = repeat_regions.loc[repeat_regions.Chrom.isin(HUMAN_CHROMOSOMES[:-1])]

# redundants = repeat_regions.ID.value_counts()[repeat_regions.ID.value_counts() != 1].index.to_list()
# for red in redundants:
# 	red_rows = repeat_regions.loc[repeat_regions.ID==red]
# 	for feature in ('Period', 'Copy_num', 'Consensus_size', 'Sequence'):
# 		if len(red_rows[feature].unique()) > 1:
# 			repeat_regions.loc[repeat_regions.ID==red, feature] = pd.NA

# assert set(repeat_regions.ID.value_counts().to_list()) == {1}, repeat_regions.ID.value_counts()
repeat_regions

,Chrom,Start,End,Period,Copy_num,Consensus_size,Sequence,ID
0,chr1,12582785,12583841,57,18.3,57,TCCATCCATCCATCCATCCATCCATCCATCCATCCCTCCCTCATCT...,chr1_12582784_12583841_0
1,chr1,12582840,12582993,4,37.8,4,TCCA,chr1_12582839_12582993_1
2,chr1,12582911,12583333,30,13.5,30,CATCCATCCATCCACTCACCTACCCCACTC,chr1_12582910_12583333_2
3,chr1,22020083,22020671,308,1.9,307,CTGTAATCCCAGCACTTTGGGAGGCCGAGGCAAGAGGATCACCTGA...,chr1_22020082_22020671_3
4,chr1,121609879,121734316,342,365.9,341,AAATATCCACTTGCAGATTCTACAAAAAGAGTGTTTCAAAACTGCT...,chr1_121609878_121734316_4
...,...,...,...,...,...,...,...,...
965506,chr22,50796716,50796758,2,21.5,2,AC,chr22_50796715_50796758_965506
965507,chr22,50799628,50800932,93,14.1,93,ATTTCAGCCATGAATCAGGCATAAATATTCTGATGGTTAATTTTAG...,chr22_50799627_50800932_965507
965508,chr22,50806959,50807016,24,2.4,24,GTCTCTGCGCCGCCGGGGACGTGT,chr22_50806958_50807016_965508
965509,chr22,50807209,50807273,29,2.2,29,GCGCCGCGCCTCTCTGCGCCTGCGCCGGC,chr22_50807208_50807273_965509


In [10]:
eq = 0
neq = 0
copy_ne_per = 0
copy_ne_con = 0
per_ne_con = 0
for _, row in repeat_regions.iterrows():
    len_by_copy = (row['End']-row['Start']+1)/row['Copy_num']
    lens = (round(len_by_copy), row['Period'], row['Consensus_size'])
    if len(set(lens)) == 1:
        eq += 1
    else:
        neq += 1
        if round(len_by_copy) != row['Period']:
            copy_ne_per += 1
        if round(len_by_copy) != row['Consensus_size']:
            copy_ne_con += 1
        if row['Period'] != row['Consensus_size']:
            per_ne_con += 1
            
        # print(round(len_by_copy, 2), lens[1:]) 
        
# print(f'{eq,}\t\t{neq:,} ({copy_ne_per:,}, {copy_ne_con:,}, {per_ne_con:,})\t\t{repeat_regions.shape[0]:,}')

In [12]:
introns = (introns.loc[introns.Chromosome.isin(HUMAN_CHROMOSOMES[:-1])]
           .rename({'Stop':'End', 'Intron_ID': 'ID'}, axis=1)
           .assign(Start = lambda df: df.Start + 1,
					Length=lambda df: df.End-df.Start + 1, 
                   	coords= lambda df: df.Chromosome + '_' + df.Start.astype(str) + '_' + df.End.astype(str))
			)
assert set(introns.ID.value_counts().to_list()) == {1}, introns.ID.value_counts()
introns[['Transcript', '', '', '', '', '', '']] = introns.ID.str.split('_', expand=True)
# introns.Strand = introns.Strand.map({'f': '+', 'r': '-'})
introns.drop(columns=[''], inplace=True)
introns

,Chromosome,Start,End,ID,Length,coords,Transcript
0,chr1,12058,12178,ENST00000450305.2_intron_0_0_chr1_12058_f,121,chr1_12058_12178,ENST00000450305.2
1,chr1,12228,12612,ENST00000450305.2_intron_1_0_chr1_12228_f,385,chr1_12228_12612,ENST00000450305.2
2,chr1,12698,12974,ENST00000450305.2_intron_2_0_chr1_12698_f,277,chr1_12698_12974,ENST00000450305.2
3,chr1,13053,13220,ENST00000450305.2_intron_3_0_chr1_13053_f,168,chr1_13053_13220,ENST00000450305.2
4,chr1,13375,13452,ENST00000450305.2_intron_4_0_chr1_13375_f,78,chr1_13375_13452,ENST00000450305.2
...,...,...,...,...,...,...,...
1396096,chr22,50785294,50788749,ENST00000701805.1_intron_2_0_chr22_50785294_f,3456,chr22_50785294_50788749,ENST00000701805.1
1396097,chr22,50788803,50788894,ENST00000701805.1_intron_3_0_chr22_50788803_f,92,chr22_50788803_50788894,ENST00000701805.1
1396098,chr22,50784022,50785172,ENST00000701632.1_intron_0_0_chr22_50784022_f,1151,chr22_50784022_50785172,ENST00000701632.1
1396099,chr22,50785294,50788749,ENST00000701632.1_intron_1_0_chr22_50785294_f,3456,chr22_50785294_50788749,ENST00000701632.1


In [ ]:
hits = []
# for chrom in ['chr1']:
for chrom in HUMAN_CHROMOSOMES[:-1]:
	print(chrom)
	repeats_sub = repeat_regions.loc[repeat_regions.Chrom == chrom]
	introns_sub = introns.loc[introns.Chromosome == chrom]

	for _, r_row in repeats_sub.iterrows():
		introns_within = introns_sub.loc[(introns_sub.Start >= r_row['Start']) & 
											(introns_sub.End <= r_row['End'])]
		if introns_within.shape[0] == 0:
			continue
		
		for _, i_row in introns_within.iterrows():
			hits_row = r_row.to_list() + i_row.to_list()
			hits.append(hits_row)

	# print('now', len(hits), 'hits')

print(len(hits))
hits_df = pd.DataFrame(hits, columns=['Chrom', 'R_start', 'R_end', 'R_period', 'R_copy_num', 'R_size', 'R_sequence', 'R_ID', '', 'I_start', 'I_end', 'I_ID', 'I_length', 'I_coords', 'Transcript'])
hits_df

chr1
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr20
chr21
chr22
chrX
chrY
2139


,Chrom,R_start,R_end,R_period,R_copy_num,R_size,R_sequence,R_ID,,I_start,I_end,I_ID,I_length,I_coords,Transcript
0,chr1,2746298,2777443,40,774.0,40,ACAGCACCCACACCCCCAGGTGAGCATCTGACAGCCTGGA,chr1_2746297_2777443,chr1,2773865,2773946,ENST00000638002.1_intron_0_0_chr1_2773865_f,82,chr1_2773865_2773946,ENST00000638002.1
1,chr1,2746298,2777443,40,774.0,40,ACAGCACCCACACCCCCAGGTGAGCATCTGACAGCCTGGA,chr1_2746297_2777443,chr1,2774067,2774229,ENST00000638002.1_intron_1_0_chr1_2774067_f,163,chr1_2774067_2774229,ENST00000638002.1
2,chr1,2746298,2777443,40,774.0,40,ACAGCACCCACACCCCCAGGTGAGCATCTGACAGCCTGGA,chr1_2746297_2777443,chr1,2774710,2774789,ENST00000638002.1_intron_2_0_chr1_2774710_f,80,chr1_2774710_2774789,ENST00000638002.1
3,chr1,2746298,2777443,40,774.0,40,ACAGCACCCACACCCCCAGGTGAGCATCTGACAGCCTGGA,chr1_2746297_2777443,chr1,2774911,2774992,ENST00000638002.1_intron_3_0_chr1_2774911_f,82,chr1_2774911_2774992,ENST00000638002.1
4,chr1,2746298,2777443,40,774.0,40,ACAGCACCCACACCCCCAGGTGAGCATCTGACAGCCTGGA,chr1_2746297_2777443,chr1,2775073,2775154,ENST00000638002.1_intron_4_0_chr1_2775073_f,82,chr1_2775073_2775154,ENST00000638002.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2134,chrY,22170046,22172335,551,4.2,548,ACACACACATGAATGGCAATAAATTTTCAAAAGCAGGAAATAACAA...,chrY_22170045_22172335,chrY,22171395,22171834,ENST00000454978.2_intron_6_0_chrY_22171395_r,440,chrY_22171395_22171834,ENST00000454978.2
2135,chrY,22414088,22416348,551,4.1,547,TTCCAACTAACATTGTATCAGGGCTCCAATTTATCTACATCCTCTC...,chrY_22414087_22416348,chrY,22414589,22415028,ENST00000250831.6_intron_6_0_chrY_22414589_f,440,chrY_22414589_22415028,ENST00000250831.6
2136,chrY,22414088,22416348,551,4.1,547,TTCCAACTAACATTGTATCAGGGCTCCAATTTATCTACATCCTCTC...,chrY_22414087_22416348,chrY,22415140,22415573,ENST00000250831.6_intron_7_0_chrY_22415140_f,434,chrY_22415140_22415573,ENST00000250831.6
2137,chrY,22414088,22416348,551,4.1,547,TTCCAACTAACATTGTATCAGGGCTCCAATTTATCTACATCCTCTC...,chrY_22414087_22416348,chrY,22414589,22415028,ENST00000470460.1_intron_5_0_chrY_22414589_f,440,chrY_22414589_22415028,ENST00000470460.1


In [ ]:
hits_df.drop('', axis=1, inplace=True)
hits_df = hits_df.assign(I_mult3 = hits_df.I_length % 3 == 0,
                         R_mult3 = hits_df.R_size % 3 == 0,
                         I_R_size_diff = hits_df.I_length - hits_df.R_size)
hits_df

,Chrom,R_start,R_end,R_period,R_copy_num,R_size,R_sequence,R_ID,I_start,I_end,I_ID,I_length,I_coords,Transcript,I_mult3,R_mult3,I_R_size_diff
0,chr1,2746298,2777443,40,774.0,40,ACAGCACCCACACCCCCAGGTGAGCATCTGACAGCCTGGA,chr1_2746297_2777443,2773865,2773946,ENST00000638002.1_intron_0_0_chr1_2773865_f,82,chr1_2773865_2773946,ENST00000638002.1,False,False,42
1,chr1,2746298,2777443,40,774.0,40,ACAGCACCCACACCCCCAGGTGAGCATCTGACAGCCTGGA,chr1_2746297_2777443,2774067,2774229,ENST00000638002.1_intron_1_0_chr1_2774067_f,163,chr1_2774067_2774229,ENST00000638002.1,False,False,123
2,chr1,2746298,2777443,40,774.0,40,ACAGCACCCACACCCCCAGGTGAGCATCTGACAGCCTGGA,chr1_2746297_2777443,2774710,2774789,ENST00000638002.1_intron_2_0_chr1_2774710_f,80,chr1_2774710_2774789,ENST00000638002.1,False,False,40
3,chr1,2746298,2777443,40,774.0,40,ACAGCACCCACACCCCCAGGTGAGCATCTGACAGCCTGGA,chr1_2746297_2777443,2774911,2774992,ENST00000638002.1_intron_3_0_chr1_2774911_f,82,chr1_2774911_2774992,ENST00000638002.1,False,False,42
4,chr1,2746298,2777443,40,774.0,40,ACAGCACCCACACCCCCAGGTGAGCATCTGACAGCCTGGA,chr1_2746297_2777443,2775073,2775154,ENST00000638002.1_intron_4_0_chr1_2775073_f,82,chr1_2775073_2775154,ENST00000638002.1,False,False,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2134,chrY,22170046,22172335,551,4.2,548,ACACACACATGAATGGCAATAAATTTTCAAAAGCAGGAAATAACAA...,chrY_22170045_22172335,22171395,22171834,ENST00000454978.2_intron_6_0_chrY_22171395_r,440,chrY_22171395_22171834,ENST00000454978.2,False,False,-108
2135,chrY,22414088,22416348,551,4.1,547,TTCCAACTAACATTGTATCAGGGCTCCAATTTATCTACATCCTCTC...,chrY_22414087_22416348,22414589,22415028,ENST00000250831.6_intron_6_0_chrY_22414589_f,440,chrY_22414589_22415028,ENST00000250831.6,False,False,-107
2136,chrY,22414088,22416348,551,4.1,547,TTCCAACTAACATTGTATCAGGGCTCCAATTTATCTACATCCTCTC...,chrY_22414087_22416348,22415140,22415573,ENST00000250831.6_intron_7_0_chrY_22415140_f,434,chrY_22415140_22415573,ENST00000250831.6,False,False,-113
2137,chrY,22414088,22416348,551,4.1,547,TTCCAACTAACATTGTATCAGGGCTCCAATTTATCTACATCCTCTC...,chrY_22414087_22416348,22414589,22415028,ENST00000470460.1_intron_5_0_chrY_22414589_f,440,chrY_22414589_22415028,ENST00000470460.1,False,False,-107


In [ ]:
hits_df.loc[hits_df.I_start==116775334]

,Chrom,R_start,R_end,R_period,R_copy_num,R_size,R_sequence,R_ID,I_start,I_end,I_ID,I_length,I_coords,Transcript,I_mult3,R_mult3,I_R_size_diff
1320,chr11,116775334,116775649,169,1.9,168,CTGTAATCCCAGCTACTCAGAAGGCTGAGGCAAGAGAATCGCTTGA...,chr11_116775333_116775649,116775334,116775502,ENST00000366405.2_intron_0_0_chr11_116775334_r,169,chr11_116775334_116775502,ENST00000366405.2,False,True,1


In [ ]:
# size_diff = []
overlap_type = []
# unit_contains = []
# unit_s_d = []
# unit_e_d = []
# unit_is = []
for _, row in hits_df.iterrows():
	units = [(x, x+row['R_size']) for x in range(row['R_start'], row['R_end'], row['R_size'])]
	units[-1] = (units[-1][0], row['R_end'])

	# if row['I_length'] > row['R_size']:
	# 	size = 'Larger intron'
	# elif row['I_length'] == row['R_size']:
	# 	size = 'Equal'
	# elif row['I_length'] < row['R_size']:
	# 	size = 'Larger repeat'

	# intron_in_unit = False
	intron_unit_start_dist = pd.NA
	intron_unit_end_dist = pd.NA
	for unit_start, unit_end in units:
		if row['I_start'] > unit_start and row['I_end'] < unit_end:
			overlap = 'Within'
			# intron_in_unit = True
			intron_unit_start_dist = row['I_start'] - unit_start
			intron_unit_end_dist = unit_end - row['I_end']
			break
		elif row['I_start'] == unit_start and row['I_end'] == unit_end:
			overlap = 'Equal'
			break
		elif unit_end > row['I_start']:
			overlap = 'Staggered'
			break

	# intron_is_unit = True if pd.notna(intron_unit_start_dist) and intron_unit_start_dist == 0 and pd.notna(intron_unit_end_dist) and intron_unit_end_dist == 0 else False

	# size_diff.append(size)
	# unit_contains.append(intron_in_unit)
	overlap_type.append(overlap)
	# unit_s_d.append(intron_unit_start_dist)
	# unit_e_d.append(intron_unit_end_dist)
	# unit_is.append(intron_is_unit)

hits_df = hits_df.assign(
						# Size_diff = size_diff,
						# I_in_R_unit = unit_contains,
						# I_R_unit_start_dist = unit_s_d,
						# I_R_unit_end_dist = unit_e_d,
						Overlap = overlap_type)
						# I_eq_R_unit = intron_is_unit)
hits_df

,Chrom,R_start,R_end,R_period,R_copy_num,R_size,R_sequence,R_ID,I_start,I_end,I_ID,I_length,I_coords,Transcript,I_mult3,R_mult3,I_R_size_diff,Overlap
0,chr1,2746298,2777443,40,774.0,40,ACAGCACCCACACCCCCAGGTGAGCATCTGACAGCCTGGA,chr1_2746297_2777443,2773865,2773946,ENST00000638002.1_intron_0_0_chr1_2773865_f,82,chr1_2773865_2773946,ENST00000638002.1,False,False,42,Staggered
1,chr1,2746298,2777443,40,774.0,40,ACAGCACCCACACCCCCAGGTGAGCATCTGACAGCCTGGA,chr1_2746297_2777443,2774067,2774229,ENST00000638002.1_intron_1_0_chr1_2774067_f,163,chr1_2774067_2774229,ENST00000638002.1,False,False,123,Staggered
2,chr1,2746298,2777443,40,774.0,40,ACAGCACCCACACCCCCAGGTGAGCATCTGACAGCCTGGA,chr1_2746297_2777443,2774710,2774789,ENST00000638002.1_intron_2_0_chr1_2774710_f,80,chr1_2774710_2774789,ENST00000638002.1,False,False,40,Staggered
3,chr1,2746298,2777443,40,774.0,40,ACAGCACCCACACCCCCAGGTGAGCATCTGACAGCCTGGA,chr1_2746297_2777443,2774911,2774992,ENST00000638002.1_intron_3_0_chr1_2774911_f,82,chr1_2774911_2774992,ENST00000638002.1,False,False,42,Staggered
4,chr1,2746298,2777443,40,774.0,40,ACAGCACCCACACCCCCAGGTGAGCATCTGACAGCCTGGA,chr1_2746297_2777443,2775073,2775154,ENST00000638002.1_intron_4_0_chr1_2775073_f,82,chr1_2775073_2775154,ENST00000638002.1,False,False,42,Staggered
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2134,chrY,22170046,22172335,551,4.2,548,ACACACACATGAATGGCAATAAATTTTCAAAAGCAGGAAATAACAA...,chrY_22170045_22172335,22171395,22171834,ENST00000454978.2_intron_6_0_chrY_22171395_r,440,chrY_22171395_22171834,ENST00000454978.2,False,False,-108,Staggered
2135,chrY,22414088,22416348,551,4.1,547,TTCCAACTAACATTGTATCAGGGCTCCAATTTATCTACATCCTCTC...,chrY_22414087_22416348,22414589,22415028,ENST00000250831.6_intron_6_0_chrY_22414589_f,440,chrY_22414589_22415028,ENST00000250831.6,False,False,-107,Staggered
2136,chrY,22414088,22416348,551,4.1,547,TTCCAACTAACATTGTATCAGGGCTCCAATTTATCTACATCCTCTC...,chrY_22414087_22416348,22415140,22415573,ENST00000250831.6_intron_7_0_chrY_22415140_f,434,chrY_22415140_22415573,ENST00000250831.6,False,False,-113,Staggered
2137,chrY,22414088,22416348,551,4.1,547,TTCCAACTAACATTGTATCAGGGCTCCAATTTATCTACATCCTCTC...,chrY_22414087_22416348,22414589,22415028,ENST00000470460.1_intron_5_0_chrY_22414589_f,440,chrY_22414589_22415028,ENST00000470460.1,False,False,-107,Staggered


In [ ]:
hits_df.Overlap.value_counts()

Overlap
Staggered    1197
Within        941
Equal           1
Name: count, dtype: int64

In [ ]:
hits_df.loc[hits_df.Overlap=='Equal']

,Chrom,R_start,R_end,R_period,R_copy_num,R_size,R_sequence,R_ID,I_start,I_end,I_ID,I_length,I_coords,Transcript,I_mult3,R_mult3,I_R_size_diff,Overlap
1320,chr11,116775334,116775649,169,1.9,168,CTGTAATCCCAGCTACTCAGAAGGCTGAGGCAAGAGAATCGCTTGA...,chr11_116775333_116775649,116775334,116775502,ENST00000366405.2_intron_0_0_chr11_116775334_r,169,chr11_116775334_116775502,ENST00000366405.2,False,True,1,Equal


In [ ]:
print(hits_df.I_mult3.value_counts())
print(hits_df.R_mult3.value_counts())
print(hits_df.Overlap.value_counts())

I_mult3
False    1452
True      687
Name: count, dtype: int64
R_mult3
False    1412
True      727
Name: count, dtype: int64
Overlap
Staggered    1197
Within        941
Equal           1
Name: count, dtype: int64


In [ ]:
for chrom in hits_df.Chrom.unique():
	print(chrom)
	chrom_t_annos = functions.get_annotations(GENCODE_v43_ANNOTATIONS_FILE, chromosome=chrom, feature='transcript')

	for tid in hits_df.loc[hits_df.Chrom==chrom, 'Transcript']:
		annos = [a for a in chrom_t_annos if a['transcript_id'].split('.')[0] == tid.split('.')[0]]
		if len(annos) != 1:
			# print(chrom, tid, annos)
			if tid == 'ENST00000706377.1':
				hits_df.loc[hits_df.Transcript==tid, 'Strand'] = '+'
				hits_df.loc[hits_df.Transcript==tid, 'Gene'] = 'PMPCA'
				hits_df.loc[hits_df.Transcript==tid, 'Gene_ID'] = 'ENSG00000165688'
				hits_df.loc[hits_df.Transcript==tid, 'Gene_type'] = 'protein_coding'
				hits_df.loc[hits_df.Transcript==tid, 'T_type'] = 'nonsense_mediated_decay'
			else:
				raise RuntimeError()

		else:
			hits_df.loc[hits_df.Transcript==tid, 'Strand'] = annos[0]['Strand']
			hits_df.loc[hits_df.Transcript==tid, 'Gene'] = annos[0]['gene_name']
			hits_df.loc[hits_df.Transcript==tid, 'Gene_ID'] = annos[0]['gene_id']
			hits_df.loc[hits_df.Transcript==tid, 'Gene_type'] = annos[0]['gene_type']
			hits_df.loc[hits_df.Transcript==tid, 'T_type'] = annos[0]['transcript_type']

hits_df

chr1
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr20
chr21
chr22
chrX
chrY


,Chrom,R_start,R_end,R_period,R_copy_num,R_size,R_sequence,R_ID,I_start,I_end,I_ID,I_length,I_coords,Transcript,I_mult3,R_mult3,I_R_size_diff,Overlap,Strand,Gene,T_type,Gene_ID,Gene_type
0,chr1,2746298,2777443,40,774.0,40,ACAGCACCCACACCCCCAGGTGAGCATCTGACAGCCTGGA,chr1_2746297_2777443_40_18,2773865,2773946,ENST00000638002.1_intron_0_0_chr1_2773865_f,82,chr1_2773865_2773946,ENST00000638002.1,False,False,42,Staggered,+,ENSG00000283259,lncRNA,ENSG00000283259.1,lncRNA
1,chr1,2746298,2777443,40,774.0,40,ACAGCACCCACACCCCCAGGTGAGCATCTGACAGCCTGGA,chr1_2746297_2777443_40_18,2774067,2774229,ENST00000638002.1_intron_1_0_chr1_2774067_f,163,chr1_2774067_2774229,ENST00000638002.1,False,False,123,Staggered,+,ENSG00000283259,lncRNA,ENSG00000283259.1,lncRNA
2,chr1,2746298,2777443,40,774.0,40,ACAGCACCCACACCCCCAGGTGAGCATCTGACAGCCTGGA,chr1_2746297_2777443_40_18,2774710,2774789,ENST00000638002.1_intron_2_0_chr1_2774710_f,80,chr1_2774710_2774789,ENST00000638002.1,False,False,40,Staggered,+,ENSG00000283259,lncRNA,ENSG00000283259.1,lncRNA
3,chr1,2746298,2777443,40,774.0,40,ACAGCACCCACACCCCCAGGTGAGCATCTGACAGCCTGGA,chr1_2746297_2777443_40_18,2774911,2774992,ENST00000638002.1_intron_3_0_chr1_2774911_f,82,chr1_2774911_2774992,ENST00000638002.1,False,False,42,Staggered,+,ENSG00000283259,lncRNA,ENSG00000283259.1,lncRNA
4,chr1,2746298,2777443,40,774.0,40,ACAGCACCCACACCCCCAGGTGAGCATCTGACAGCCTGGA,chr1_2746297_2777443_40_18,2775073,2775154,ENST00000638002.1_intron_4_0_chr1_2775073_f,82,chr1_2775073_2775154,ENST00000638002.1,False,False,42,Staggered,+,ENSG00000283259,lncRNA,ENSG00000283259.1,lncRNA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2134,chrY,22170046,22172335,551,4.2,548,ACACACACATGAATGGCAATAAATTTTCAAAAGCAGGAAATAACAA...,chrY_22170045_22172335_548_573126,22171395,22171834,ENST00000454978.2_intron_6_0_chrY_22171395_r,440,chrY_22171395_22171834,ENST00000454978.2,False,False,-108,Staggered,-,RBMY1F,protein_coding,ENSG00000169800.14,protein_coding
2135,chrY,22414088,22416348,551,4.1,547,TTCCAACTAACATTGTATCAGGGCTCCAATTTATCTACATCCTCTC...,chrY_22414087_22416348_547_573208,22414589,22415028,ENST00000250831.6_intron_6_0_chrY_22414589_f,440,chrY_22414589_22415028,ENST00000250831.6,False,False,-107,Staggered,+,RBMY1J,protein_coding,ENSG00000226941.9,protein_coding
2136,chrY,22414088,22416348,551,4.1,547,TTCCAACTAACATTGTATCAGGGCTCCAATTTATCTACATCCTCTC...,chrY_22414087_22416348_547_573208,22415140,22415573,ENST00000250831.6_intron_7_0_chrY_22415140_f,434,chrY_22415140_22415573,ENST00000250831.6,False,False,-113,Staggered,+,RBMY1J,protein_coding,ENSG00000226941.9,protein_coding
2137,chrY,22414088,22416348,551,4.1,547,TTCCAACTAACATTGTATCAGGGCTCCAATTTATCTACATCCTCTC...,chrY_22414087_22416348_547_573208,22414589,22415028,ENST00000470460.1_intron_5_0_chrY_22414589_f,440,chrY_22414589_22415028,ENST00000470460.1,False,False,-107,Staggered,+,RBMY1J,protein_coding_CDS_not_defined,ENSG00000226941.9,protein_coding


In [ ]:
hits_df.to_csv('data/repeat_regions_x_introns.tsv', sep='\t', index=False)

In [ ]:
intron_coords = []
with open('results/repeat_regions_x_introns.bed', 'w') as w:
	w.write('browser hide all\n')
	w.write('browser full knownGene\n')
	w.write('browser squish simpleRepeat\n')
	w.write('track name="Introns in Tandem Repeats"	type=bed	db=hg38	visibility=2	itemRgb=on\n')
	for _, row in hits_df.iterrows():
		color = '255,153,0' if row['R_mult3'] else '166,166,166'
		track_row = f"{row['Chrom']}\t{row['I_start']-1}	{row['I_end']}	{row['R_size']}bp_repeat	0	.	0	0	{color}\n"
		
		coords = (row['Chrom'], row['I_start'], row['I_end'])
		if coords not in intron_coords:
			intron_coords.append(coords)
		else:
			track_row = '#' + track_row

		w.write(track_row)